### one time installations

In [1]:
# !gdown --id 1pb7gEkctrVrJA79EAIo7H7nuzD6uV1fW
# !gdown --id 1oIeAE9HXXKWPcYa-AZ0ht5ef6sKe_Vh_
# !gdown --id 10rAuIDvsYR2yDiCqP7GmYGPc-UmtLbJb

In [2]:
# !pip install --quiet transformers
# !pip install --quiet datasets 
# !pip install --quiet SentencePiece
# !pip install --quiet pytorch-lightning

### libraries

In [27]:
import numpy as np
import pandas as pd
import sklearn

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader

from transformers import AutoModel, AutoTokenizer

### hyper parameters

In [4]:
class hyperparameters:
    # tokenizer
    tokenizer_name = "deepset/xlm-roberta-large-squad2" # model_name
    max_len = 384 # maximum length of context and question in a datapoint
    overlap_len = 128 # overlap between two parts of the context when it is split

In [5]:
hyperparams = hyperparameters()

### data - understanding the pipeline

In [75]:
tokenizer = AutoTokenizer.from_pretrained(hyperparams.tokenizer_name)

In [70]:
train_df = pd.read_csv('train.csv', encoding='utf-8')
# test_df = pd.read_csv('test.csv')
# sample_df = pd.read_csv('sample_submission.csv')

In [71]:
train_df = sklearn.utils.shuffle(train_df, random_state=4).reset_index(drop=True)

In [73]:
train_df = train_df.loc[:2]
# train_df

In [74]:
# def convert_answers(row):
#     return {'answer_start': [row[0]], 'text': [row[1]]}

# train_df['answers'] = train_df[['answer_start', 'answer_text']].apply(convert_answers, axis=1)

In [58]:
for idx in range(len(train_df)):
    if train_df.loc[idx,'language'] == "hindi" and len(tokenizer(train_df.loc[idx, 'context'], train_df.loc[idx, 'question'])['input_ids']) > hyperparams.max_len:
        break

In [59]:
# print(train_df.loc[idx, 'question'])
# print(train_df.loc[idx, 'context'])

In [90]:
out = tokenizer(
    list(train_df['question']), list(train_df['context']),
    max_length=hyperparams.max_len, 
    truncation='only_second',
    stride=hyperparams.overlap_len,
    return_overflowing_tokens=True,
    return_offsets_mapping=True,
    padding=True,
)

In [91]:
print(out.keys())
print(len(out['input_ids']), len(out['offset_mapping']))

dict_keys(['input_ids', 'attention_mask', 'offset_mapping', 'overflow_to_sample_mapping'])
62 62


In [95]:
print(len(out['offset_mapping']))
# for i in range(len(out['offset_mapping'])):
#     print(len(out.sequence_ids(i)))

62


In [96]:
print(tokenizer.bos_token, tokenizer.padding_side)

<s> right


In [97]:
print(len(out['input_ids']))

62


In [98]:
# print(tokenizer.decode(out['input_ids'][0]))
# print(tokenizer.decode(out['input_ids'][1]))
# print(tokenizer.decode(out['input_ids'][2]))

In [99]:
map_sample2original = out.pop('overflow_to_sample_mapping')
print(map_sample2original)

[0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2]


In [35]:
data_df = train_df

In [36]:
data_df.loc[:, 'context'] = data_df.loc[:, 'context'].apply(lambda sen : str(sen).strip())
data_df.loc[:, 'question'] = data_df.loc[:, 'question'].apply(lambda sen : str(sen).strip())

In [49]:
print(data_df['question'].values, len(data_df['question'].values))

['ரேடியம் எப்போது கண்டுபிடிக்கப்பட்டது?'
 'हेलेन केलर की मृत्यु किस वर्ष में हुई थी?'
 'खाद्य एवं औषधि प्रशासन का मुख्यालय कहाँ पर है?'
 'क़ुस्तुंतुनिया शहर की स्थापना किस सम्राट ने की थी?'
 'दिल्ली का क्षेत्रफल कितना है?' 'जापान का सबसे पहला राजा कौन था?'
 '२०१६ ग्रीष्मकालीन ओलम्पिक में कितने राष्ट्रों ने भाग लिया था?'
 'ராமானுஜம் எங்கு பிறந்தார்?' 'ஆஸ்திரேலியாவில் எத்தனை மாநிலங்கள் உள்ளன?'
 'विश्व पर्यावरण दिवस किस दिन मनाया जाता है?'
 'ओटो एडुअर्ड लिओपोल्ड बिस्मार्क को किस वर्ष पेरिस में राजदूत बनाया गया?'
 'किसने १९३२ में हिन्दुस्तान टाइम्स में कार्टून बनाने प्रारम्भ किए?'
 'अन्ना हजारे का जन्म कहाँ हुआ था?'
 'अभिनेत्री ब्लेक लिवली की माता का क्या नाम है?'
 'இசுரேல் நாட்டின் பரப்பளவு என்ன?' 'रोमानिया की आधिकारिक भाषा क्या है?'
 'யானையின் சராசரி ஆயுட்காலம் எவ்வளவு?'
 'आई एन एस विक्रमादित्य को किस वर्ष भारतीय नौसेना में शामिल कर लिया गया?'
 'உலகில் எத்தனை பாலூட்டி இனங்கள் உள்ளன?' 'அணு உலையை கண்டுபிடித்தவர் யார்?'
 'सलमा हायेक के पति का नाम क्या है?'] 21


In [42]:
data_tokenized = tokenizer(
    list(data_df['question']), list(data_df['context']),
    max_length=hyperparams.max_len, 
    truncation='only_second',
    stride=hyperparams.overlap_len,
    return_overflowing_tokens=True,
    return_offsets_mapping=True,
    padding=True,
)

In [47]:
print(len(data_df), len(data_tokenized['input_ids']))

21 290


### Data pipeline

In [ ]:
def process_chaii(data_df, tokenizer, train, test):
    # strip trailing and leading whitespaces in context, question, and (answer_text)?
    data_df.loc[:, 'context'] = data_df.loc[:, 'context'].apply(lambda sen : str(sen).strip())
    data_df.loc[:, 'question'] = data_df.loc[:, 'question'].apply(lambda sen : str(sen).strip())
    if test == False:
        data_df.loc[:, 'answer_text'] = data_df.loc[:, 'answer_text'].apply(lambda sen : str(sen).strip())
    
    

In [ ]:
class chaii_ka_data(Dataset):
    def __init__(self, data_df, tokenizer, train=True, test=False):
        super(chaii_ka_data, self).__init__()
        '''
            train = True means train, train = False means val set; test = True means test set (without labels)
            data_df is the pandas dataframe containing context, question, ...        
        '''
        self.train = train
        self.test = test
        
        # tokenize data samples context;question, and create new samples if overflow
        self.data = process_chaii(data_df, tokenizer, train, test)
    
    def __getitem__(self, index):
        pass
    
    def __len__(self):
        pass

### model

### training

### evaluation

### references
1. https://github.com/huggingface/notebooks/blob/master/examples/question_answering.ipynb
2. https://huggingface.co/transformers/internal/tokenization_utils.html#transformers.tokenization_utils_base.PreTrainedTokenizerBase.__call__